### Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import os
import pathlib
from utils import *
import random
import numpy as np
import cv2

In [ ]:
X, y = read_images()
X = crop_images(X)
X = resize_and_rescale(X)
X_train, y_train, X_test, y_test = split_data(X, y)
X_train, y_train = augment_data(X_train,y_train)

X_train = np.array([[[[x,x,x]  for x in x2] for x2 in x1] for x1 in X_train])
X_test = np.array([[[[x,x,x]  for x in x2] for x2 in x1] for x1 in X_test])


In [ ]:
vgg16 = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet")

vgg16.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = tf.keras.applications.vgg16.preprocess_input(
    inputs, data_format=None)

x = vgg16(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(4096, activation="relu")(x)
x = tf.keras.layers.Dense(4096, activation="relu")(x)
outputs = tf.keras.layers.Dense(2, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    epochs=10,
                    validation_data=(X_test, y_test))

In [ ]:
model.evaluate(X_test, y_test)